In [1]:
'''
Adds in stellar radii for systems whose stellar radii are missing in the exoarchive catalog but not in Gaia data release 2.
Also, the code classifies any system whose exoplanets were detected by more than one method as 'Mixed'.
'''

"\nAdds in stellar radii for systems whose stellar radii are missing in the exoarchive catalog but not in Gaia data release 2.\nAlso, the code classifies any system whose exoplanets were detected by more than one method as 'Mixed'.\n"

In [2]:
import csv
import numpy as np

from astropy import constants as apc

MJUPITER = apc.M_jup.value
MEARTH = apc.M_earth.value
RJUPITER = apc.R_jup.value
REARTH = apc.R_earth.value

In [3]:
class System:
    def __init__(self, sysid=None):
        self.sysid = sysid

In [4]:
#reading in file
my_file = 'C:/Users/djhoo/Documents/Oviraptor-master/catalogues/oviraptor_crossmatch_catalog.csv'  
#my_file = "/Users/research/projects/oviraptor/catalogues/oviraptor_crossmatch_catalog.csv"

raw_data = []
with open(my_file) as infile:
    reader = csv.reader(infile)
    
    for row in reader:
        raw_data.append(row)
        
KEYS = raw_data[0]
VALUES = raw_data[1:]

def get_key(keyname, keys = KEYS, values = VALUES):
    i = keys.index(keyname)
    arr = []
    for row in values:
        arr.append(row[i])
    return np.array(arr)

d = {}
d['starname'] = get_key('hostname')
d['rp'] = get_key('pl_rade')
d['rerr1'] = get_key('pl_radeerr1')
d['rerr2'] = get_key('pl_radeerr2')
d['mp'] = get_key('pl_bmasse')
d['merr1'] = get_key('pl_bmasseerr1')
d['merr2'] = get_key('pl_bmasseerr2')
d['rs'] = get_key('st_rad')
d['ms'] = get_key('st_mass')
d['p'] = get_key('pl_orbper')
d['disc_method'] = get_key('discoverymethod')
d['temp'] = get_key('st_teff')
d['mag'] = get_key('sy_vmag')
d['ref'] = get_key('st_refname')
d['instrument'] = get_key('disc_facility')

print('Number of planets in crossmatched catalog: %d' %len(d['starname']))

Number of planets in crossmatched catalog: 4097


In [5]:
#def convert_masses(data):
#    masses = np.zeros(len(data))
#    for i in range(len(masses)):
#        if data[i] != '':
#            masses[i] = float(data[i])*MJUPITER/MEARTH
#    return masses

#def convert_radii(data):
#    radii = np.zeros(len(data))
#    for i in range(len(radii)):
#        if data[i] != '':
#            radii[i] = float(data[i])*RJUPITER/REARTH
#    return radii

In [6]:
#my_file = 'C:/Users/djhoo/Documents/maiasaurus-master/catalogues/exoarchive_ipac_confirmed_20200507.csv'
#my_file = "/Users/research/projects/oviraptor/catalogues/exoarchive_ipac_confirmed_20201008.csv"


#raw_data = []
#with open(my_file) as infile:
#    reader = csv.reader(infile)
    
#    for row in reader:
#        raw_data.append(row)
        
#KEYS = raw_data[77]
#VALUES = raw_data[78:]

#def get_key(keyname, keys = KEYS, values = VALUES):
#    i = keys.index(keyname)
#    arr = []
#    for row in values:
#        arr.append(row[i])
#    return arr

#names = get_key('pl_hostname')     
#rs = get_key('st_rad')
#ms = get_key('st_mass')

#print('Number of Exoarchive planets: %d' %len(names))

#'''
#if exoarchive stellar mass/radius is something and corresponding oviraptor stellar mass/radius is nothing:
#replace d['ms'] or d['rs'] with corresponding value
#'''

#for i in range(len(names)):
#    if ms[i] != '' and d['ms'][d['starname'] == names[i]].tolist() == ['']*len(d['ms'][d['starname'] == names[i]]):
#        for j in range(len(d['ms'])):
#            if d['starname'][j] == names[i]:
#                d['ms'][j] = ms[i]
#    if rs[i] != '' and d['rs'][d['starname'] == names[i]].tolist() == ['']*len(d['rs'][d['starname'] == names[i]]):
#        for j in range(len(d['rs'])):
#            if d['starname'][j] == names[i]:
#                d['rs'][j] = rs[i]

In [7]:
#npl = np.array(get_key('sy_pnum'), dtype="int")
#sdm = np.copy(d["disc_method"])

#for i, s in enumerate(np.unique(d["starname"])):
#    use = d["starname"] == s
    
#    disc_method = d["disc_method"][use]
    
#    all_same = True
#    for dm in disc_method:
#        if dm != disc_method[0]:
#            all_same = False
            
#    if all_same == False:
#        sdm[use] = "Mixed"
        
#d["system_disc_method"] = np.copy(sdm)

In [8]:
def check_uniformity(detection_methods):
    default_disc = detection_methods[0]
    status = True
    for i in range(len(detection_methods)):
        if detection_methods[i] != default_disc:
            status = False
    return status

d['system_disc_method'] = []

for i in range(len(d['starname'])):
    if check_uniformity(d['disc_method'][d['starname']==d['starname'][i]]) == False:
        d['system_disc_method'].append('Mixed')
    else:
        d['system_disc_method'].append(d['disc_method'][i])

In [9]:
#for i in range(len(d['starname'])):
#    # replicating values of stellar radius for planets within the same system
#    radii = d['rs'][d['starname'] == d['starname'][i]]
#    if len(radii[radii != '']) > 0:
#        true_radius = radii[radii != ''][0]
#        d['rs'][i] = true_radius
#        
#    # doing basically the same thing but for stellar masses
#    mass = d['ms'][d['starname'] == d['starname'][i]]
#    if len(mass[mass != '']) > 0:
#        true_mass = mass[mass != ''][0]
#        d['ms'][i] = true_mass

In [10]:
gaia_stars = {}
gaia_stars['stars'] = ['HD 136352', 'HD 141399', 'HD 160691', 'HD 20781', 'HD 20794', 'HD 27984', 'HD 31527', \
                      'HD 40307', 'tau Cet']
gaia_stars['radii'] = [1.0041078, 1.3874089, 1.3901234, 0.8336087, 0.83483344, 0.89176697, 1.0736936, 1.3310363, \
                      0.793]  

#all of these radii are taken from the Gaia archive
#except for tau Cet, which was taken from Dietrich and Apai (2020)

for i in range(len(d['starname'])):
    if d['starname'][i] in gaia_stars['stars']:
        d['rs'][i] = gaia_stars['radii'][gaia_stars['stars'] == d['starname'][i]]

In [11]:
#manually add in exoplanet radii and radii errors from HIP 41378
#This comes from Berardo et. al. (2019)
#link: https://ui.adsabs.harvard.edu/abs/2019AJ....157..185B/abstract
#all values were calculated manually starting from the radii of the star, rp/rs, and the uncertainties
#note that the order of the planets in the oviraptor catalog is c, d, e, b, f

hip = {}
hip['planet_radius'] = [2.936, 3.758, 5.411, 2.705, 9.691]   
hip['rerr_lower'] = [0.55, 0.14, 0.17, 0.084, 0.23] 
hip['rerr_upper'] = [2.6, 0.12, 0.21, 0.17, 0.23]

d['rp'][d['starname'] == 'HIP 41378'] = hip['planet_radius']
d['rerr1'][d['starname'] == 'HIP 41378'] = hip['rerr_upper']
d['rerr2'][d['starname'] == 'HIP 41378'] = hip['rerr_lower']

In [12]:
#manually add in stellar mass and radius for GJ 163
#link: https://ui.adsabs.harvard.edu/abs/2013A%26A...556A.110B/abstract
#another link: https://ui.adsabs.harvard.edu/abs/2013A%26A...556A.111T/abstract

d['ms'][d['starname'] == 'GJ 163'] = [0.40]*len(d['starname'][d['starname'] == 'GJ 163'])
d['rs'][d['starname'] == 'GJ 163'] = [0.40]*len(d['starname'][d['starname'] == 'GJ 163'])

#doing the same thing for GJ 180: https://arxiv.org/pdf/2001.02577.pdf
#assuming the stellar radius for GJ 180 based on the fact that it is a M2V dwarf

d['ms'][d['starname'] == 'GJ 180'] = [0.43]*len(d['starname'][d['starname'] == 'GJ 180'])
d['rs'][d['starname'] == 'GJ 180'] = [0.44]*len(d['starname'][d['starname'] == 'GJ 180'])

In [13]:
npl = np.array(get_key('sy_pnum'), dtype="int")
Rstar = np.array(d['rs'])
Mstar = np.array(d['ms'])
sdm = np.array(d['system_disc_method'])
starname = np.array(d['starname'])

print("\n\nMissing RADIUS")
print(np.unique(starname[(Rstar == '')*(npl > 2)]))


print("\n\nMissing MASS")
print(np.unique(starname[(Mstar == '')*(npl > 2)]))



Missing RADIUS
['GJ 433' 'GJ 667 C' 'HD 133131 A' 'HD 133131 B' 'HD 27894' 'HD 37124'
 'HD 69830']


Missing MASS
['Kepler-257' 'Kepler-359']


In [14]:
'''
The data entries are the following:

- Starname - name of host star
- Rp - radius of exoplanet (in Earth radii)
- Mp - mass of exoplanet (in Earth masses)
- P - period of exoplanet (in days)
- Rs - radius of host star (in solar radii)
- Ms - mass of host star (in solar masses)
- Detection Method - detection method (Radial Velocity, Transit, or Mixed)
- System Detection Method - the detection method of a system as a whole
- Temp - temperature of the star (in Kelvin)
- Mag - visual magnitude of the star
- Instrument - discovery instrument of the star
- Ref - reference paper
'''

output_heading = ['Starname', 'Rp', 'R_upper', 'R_lower', 'Mp', 'M_upper', 'M_lower', 'P', 'Rs', 'Ms', \
                  'Detection Method', 'System Detection Method', 'Temp', 'Mag', 'Instrument', 'Ref']
myData = [output_heading]
for i in range(len(d['starname'])):
    arr = []
    arr.append(d['starname'][i])
    arr.append(d['rp'][i])
    arr.append(d['rerr1'][i])
    arr.append(d['rerr2'][i])
    arr.append(d['mp'][i])
    arr.append(d['merr1'][i])
    arr.append(d['merr2'][i])
    arr.append(d['p'][i])
    arr.append(d['rs'][i])
    arr.append(d['ms'][i])
    arr.append(d['disc_method'][i])
    arr.append(d['system_disc_method'][i])
    arr.append(d['temp'][i])
    arr.append(d['mag'][i])
    arr.append(d['instrument'][i])
    arr.append(d['ref'][i])
    myData.append(arr)

outputFile = open('C:/Users/djhoo/Documents/Oviraptor-master/catalogues/oviraptor_master.csv', 'w', newline='')
#outputFile = open("/Users/research/projects/oviraptor/catalogues/oviraptor_master.csv", "w", newline='')

with outputFile:
    writer = csv.writer(outputFile)
    writer.writerows(myData)
print('Writing Complete!')

Writing Complete!
